In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

C:\Users\Admin\AppData\Local\Temp\ipykernel_13052\2513960026.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Extracting features of 2022 movies from Wikipedia

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2022"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

C:\Users\Admin\AppData\Local\Temp\ipykernel_13052\2803005693.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df1 = pd.read_html(str(tables[0]))[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_13052\2803005693.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df2 = pd.read_html(str(tables[1]))[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_13052\2803005693.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df3 = pd.read_html(str(tables[2]))[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_13052\2803005693.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version.

In [7]:
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [8]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J A N U A R Y,7.0,The 355,Universal Pictures / Freckle Films / FilmNatio...,Simon Kinberg (director/screenplay); Theresa R...,[2]
1,J A N U A R Y,7.0,The Legend of La Llorona,Saban Films / Ageless Pictures,Patricia Harris Seeley (director); José Prende...,[3]
2,J A N U A R Y,7.0,The Commando,Saban Films / Premiere Entertainment,Asif Akbar (director); Koji Steven Sakai (scre...,[4]
3,J A N U A R Y,7.0,American Siege,Vertical Entertainment,Edward John Drake (director/screenplay); Timot...,[5]
4,J A N U A R Y,14.0,Scream,Paramount Pictures / Spyglass Media Group / Ra...,"Matt Bettinelli-Olpin, Tyler Gillett (director...",[6]
...,...,...,...,...,...,...
312,D E C E M B E R,30.0,"Alice, Darling",Lionsgate / Elevation Pictures,Mary Nighy (director); Alanna Francis (screenp...,[261]
313,D E C E M B E R,NaN,NaN,NaN,NaN,NaN
314,D E C E M B E R,NaN,NaN,NaN,NaN,NaN
315,D E C E M B E R,NaN,NaN,NaN,NaN,NaN


In [9]:
df_2022 = df[['Title','Cast and crew']]

In [10]:
df_2022

,Title,Cast and crew
0,The 355,Simon Kinberg (director/screenplay); Theresa R...
1,The Legend of La Llorona,Patricia Harris Seeley (director); José Prende...
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...
3,American Siege,Edward John Drake (director/screenplay); Timot...
4,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director..."
...,...,...
312,"Alice, Darling",Mary Nighy (director); Alanna Francis (screenp...
313,NaN,NaN
314,NaN,NaN
315,NaN,NaN


In [11]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = 'YOUR_API_KEY'

In [12]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    try:
        genres = []
        result = tmdb_movie.search(x)
        if not result:
            return np.NaN
        else:
            movie_id = result[0].id
            response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id, tmdb.api_key))
            data_json = response.json()
            if data_json['genres']:
                genre_str = " " 
                for i in range(0, len(data_json['genres'])):
                    genres.append(data_json['genres'][i]['name'])
                return genre_str.join(genres)
            else:
                return np.NaN
    except Exception as e:
        return np.NaN


In [13]:
df_2022['genres'] =df_2022['Title'].map(lambda x: get_genre(str(x)))

C:\Users\Admin\AppData\Local\Temp\ipykernel_13052\3074315850.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['genres'] =df_2022['Title'].map(lambda x: get_genre(str(x)))


In [14]:
df_2022

,Title,Cast and crew,genres
0,The 355,Simon Kinberg (director/screenplay); Theresa R...,Action Thriller
1,The Legend of La Llorona,Patricia Harris Seeley (director); José Prende...,Family Animation Fantasy Horror
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller
3,American Siege,Edward John Drake (director/screenplay); Timot...,Action Adventure Thriller
4,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Horror Mystery Thriller
...,...,...,...
312,"Alice, Darling",Mary Nighy (director); Alanna Francis (screenp...,Thriller Drama Romance
313,NaN,NaN,Drama Crime
314,NaN,NaN,Drama Crime
315,NaN,NaN,Drama Crime


In [15]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [16]:
df_2022['director_name'] = df_2022['Cast and crew'].map(lambda x: get_director(str(x)))


C:\Users\Admin\AppData\Local\Temp\ipykernel_13052\3644665622.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['director_name'] = df_2022['Cast and crew'].map(lambda x: get_director(str(x)))


In [17]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [18]:
df_2022['actor_1_name'] = df_2022['Cast and crew'].map(lambda x: get_actor1(str(x)))


C:\Users\Admin\AppData\Local\Temp\ipykernel_13052\2887556167.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['actor_1_name'] = df_2022['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [19]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [20]:
df_2022['actor_2_name'] = df_2022['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [21]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [22]:
df_2022['actor_3_name'] = df_2022['Cast and crew'].map(lambda x: get_actor3(str(x)))


In [23]:
df_2022

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The 355,Simon Kinberg (director/screenplay); Theresa R...,Action Thriller,Simon Kinberg,Jessica Chastain,Penélope Cruz,Fan Bingbing
1,The Legend of La Llorona,Patricia Harris Seeley (director); José Prende...,Family Animation Fantasy Horror,Patricia Harris Seeley,Autumn Reeser,Danny Trejo,Antonio Cupo
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller,Asif Akbar,Mickey Rourke,Michael Jai White,NaN
3,American Siege,Edward John Drake (director/screenplay); Timot...,Action Adventure Thriller,Edward John Drake,Timothy V. Murphy,Bruce Willis,Rob Gough
4,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Horror Mystery Thriller,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega
...,...,...,...,...,...,...,...
312,"Alice, Darling",Mary Nighy (director); Alanna Francis (screenp...,Thriller Drama Romance,Mary Nighy,Anna Kendrick,Kaniehtiio Horn,Charlie Carrick
313,NaN,NaN,Drama Crime,nan,nan,NaN,NaN
314,NaN,NaN,Drama Crime,nan,nan,NaN,NaN
315,NaN,NaN,Drama Crime,nan,nan,NaN,NaN


In [24]:
df_2022 = df_2022.rename(columns={'Title':'movie_title'})

In [25]:
new_df22 = df_2022.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [26]:
new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Simon Kinberg,Jessica Chastain,Penélope Cruz,Fan Bingbing,Action Thriller,The 355
1,Patricia Harris Seeley,Autumn Reeser,Danny Trejo,Antonio Cupo,Family Animation Fantasy Horror,The Legend of La Llorona
2,Asif Akbar,Mickey Rourke,Michael Jai White,NaN,Action Crime Thriller,The Commando
3,Edward John Drake,Timothy V. Murphy,Bruce Willis,Rob Gough,Action Adventure Thriller,American Siege
4,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,Scream
...,...,...,...,...,...,...
312,Mary Nighy,Anna Kendrick,Kaniehtiio Horn,Charlie Carrick,Thriller Drama Romance,"Alice, Darling"
313,nan,nan,NaN,NaN,Drama Crime,NaN
314,nan,nan,NaN,NaN,Drama Crime,NaN
315,nan,nan,NaN,NaN,Drama Crime,NaN


In [27]:
new_df22['comb'] = new_df22['actor_1_name'] + ' ' + new_df22['actor_2_name'] + ' '+ new_df22['actor_3_name'] + ' '+ new_df22['director_name'] +' ' + new_df22['genres']

In [28]:
new_df22.isna().sum()

director_name     0
actor_1_name      0
actor_2_name     11
actor_3_name     35
genres            1
movie_title       4
comb             36
dtype: int64

In [29]:
new_df22 = new_df22.dropna(how='any')

In [30]:
new_df22.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [31]:
new_df22['movie_title'] = new_df22['movie_title'].str.lower()

C:\Users\Admin\AppData\Local\Temp\ipykernel_13052\2732328091.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df22['movie_title'] = new_df22['movie_title'].str.lower()


In [32]:
new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Simon Kinberg,Jessica Chastain,Penélope Cruz,Fan Bingbing,Action Thriller,the 355,Jessica Chastain Penélope Cruz Fan Bingbing Si...
1,Patricia Harris Seeley,Autumn Reeser,Danny Trejo,Antonio Cupo,Family Animation Fantasy Horror,the legend of la llorona,Autumn Reeser Danny Trejo Antonio Cupo Patrici...
3,Edward John Drake,Timothy V. Murphy,Bruce Willis,Rob Gough,Action Adventure Thriller,american siege,Timothy V. Murphy Bruce Willis Rob Gough Edwar...
4,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,scream,Melissa Barrera Mason Gooding Jenna Ortega Mat...
5,"Derek Drymon, Jennifer Kluska",Andy Samberg,Selena Gomez,Kathryn Hahn,Animation Comedy Family Adventure Fantasy,hotel transylvania: transformania,Andy Samberg Selena Gomez Kathryn Hahn Derek D...
...,...,...,...,...,...,...,...
308,Kasi Lemmons,Naomi Ackie,Stanley Tucci,Ashton Sanders,Music History Drama,whitney houston: i wanna dance with somebody,Naomi Ackie Stanley Tucci Ashton Sanders Kasi ...
309,Scott Cooper,Christian Bale,Harry Melling,Gillian Anderson,Thriller Crime Horror Mystery,the pale blue eye,Christian Bale Harry Melling Gillian Anderson ...
310,Sarah Polley,Rooney Mara,Claire Foy,Jessie Buckley,Drama Crime,women talking,Rooney Mara Claire Foy Jessie Buckley Sarah Po...
311,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller,Comedy Drama,a man called otto,Tom Hanks Mariana Treviño Rachel Keller Marc F...


In [33]:
old_df = pd.read_csv('2021_and_prior.csv')

In [34]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6428,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Action Comedy Adventure Thriller,the king's man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...
6429,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War,the tragedy of macbeth,Denzel Washington Frances McDormand Bertie Car...
6430,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,Drama Romance,a journal for jordan,Michael B. Jordan Chanté Adams Jalon Christian...
6431,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama,american underdog,Zachary Levi Anna Paquin Dennis Quaid Erwin br...


In [35]:
final_df = pd.concat([old_df, new_df22], ignore_index=True)

In [36]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6709,Kasi Lemmons,Naomi Ackie,Stanley Tucci,Ashton Sanders,Music History Drama,whitney houston: i wanna dance with somebody,Naomi Ackie Stanley Tucci Ashton Sanders Kasi ...
6710,Scott Cooper,Christian Bale,Harry Melling,Gillian Anderson,Thriller Crime Horror Mystery,the pale blue eye,Christian Bale Harry Melling Gillian Anderson ...
6711,Sarah Polley,Rooney Mara,Claire Foy,Jessie Buckley,Drama Crime,women talking,Rooney Mara Claire Foy Jessie Buckley Sarah Po...
6712,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller,Comedy Drama,a man called otto,Tom Hanks Mariana Treviño Rachel Keller Marc F...


In [37]:
final_df.to_csv('2022_and_prior.csv',index=False)